In [1]:
import sys
paths = [
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/config",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/ds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_intrinio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/web",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_portfolio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium/api",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/estypes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esmappings",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esindexes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/utils",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_quandl",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_snp",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/public",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_barchart",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_redis",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/models",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_twitter",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_edgar",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_fred",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_oecd",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_websocket",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/symbol_mapping",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cryptocurrency",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/user",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cbonds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_brokers"
]

In [2]:
sys.path += paths
# sys.path.append("C:\\Users\\tbui\\Documents\\Stansberry-Terminal-MkI\\services\\screener_sbanalysis_api\\")

In [3]:

from pandas.io import sql as psql

In [4]:
from lib.connected_db import PgsqlExecutor

In [5]:
research_db = {
        "host": "dev-df-postgresql.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "df_user",
        "credentials": "Piq914sT",
        "database": "datafactory"
      }
snpsource = {
        "host": "snp-instance-1-us-east-1b.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "snpx_read",
        "credentials": "Nsp563Xz",
        "database": "xpressfeed"
      }

In [6]:
db = PgsqlExecutor(snpsource)
model_db = PgsqlExecutor(research_db)

# Long investment - consistent growth up

In [7]:
def get_long_stock_data(year):
    sql = """

    SELECT id, symbol, exchangesymbol, tradingitemid, price, pricing_date, 
    average_growth_rate, adjusted_growth, is_yearly_consistent_rate, 
    dcf, market_dcf, adjusted_dcf, adjusted_dcf_to_price, year_min_price, 
    year_max_price, year_end_price, end_pricing_date, dcf_ai_score, 
    predict_direction, fiscal_year

    from model.dcf_values_rank
    where fiscal_year = %s
    and adjusted_dcf_to_price != 'nan'
    order by adjusted_dcf_to_price desc
    limit 30

    """
    data = model_db.fetch_all_rows(sql, args=[year])
    return data

def get_short_stock_data(year):
    """
    You don't want to short pennies stock, you only care when stock > 10
    """
    sql = """

    SELECT id, symbol, exchangesymbol, tradingitemid, price, pricing_date, 
    average_growth_rate, adjusted_growth, is_yearly_consistent_rate, 
    dcf, market_dcf, adjusted_dcf, adjusted_dcf_to_price, year_min_price, 
    year_max_price, year_end_price, end_pricing_date, dcf_ai_score, 
    predict_direction, fiscal_year

    from model.dcf_values_rank
    where fiscal_year = %s
    and adjusted_dcf_to_price != 'nan'
    and dcf_ai_score < -30
    and price > 10
    order by adjusted_dcf_to_price
    limit 40

    """
    data = model_db.fetch_all_rows(sql, args=[year])
    return data
    


In [8]:
REPORT = """
**********- Score: {score} - {symbol} - {exchange}: {start_price} on {start_date} - *********
End date: {end_date} - price: {end_price} - min: {min_price} - max: {max_price}
Profit: {perf_profit} - max potential: {perf_potential} 
Average growth rate: {avg_growth}, consistent: {is_consistent}, project growth: {adj_growth}
DCF: {dcf}, market_dcf: {market_dcf}, adjusted_dcf: {adj_dcf}, dcf_to_price : {dcf_to_price}

"""
import numpy

def report_all(year):
    profits = []
    potentials = []
    data = get_long_stock_data(year)
    for row in data.query_data:
        profit = round(
            (row.year_end_price - row.price) * 100 / row.price
            ,1)
        potential = round(
            (row.year_max_price - row.price) * 100 / row.price
            ,1)
        profits.append(profit)
        potentials.append(potential)

        report_statement = REPORT.format(
            score=row.dcf_ai_score,
            symbol = row.symbol,
            exchange = row.exchangesymbol,
            start_price = row.price,
            start_date = row.pricing_date,
            end_date=row.end_pricing_date,
            end_price=row.year_end_price,
            min_price=row.year_min_price,
            max_price=row.year_max_price,
            perf_profit=profit,
            perf_potential=potential,
            avg_growth = row.average_growth_rate,
            is_consistent = row.is_yearly_consistent_rate,
            adj_growth = row.adjusted_growth,
            dcf=row.dcf,
            market_dcf=row.market_dcf,
            adj_dcf=row.adjusted_dcf,
            dcf_to_price =row.adjusted_dcf_to_price




        )

        print(report_statement)

    print("*** Average Profit: {}% , Potential: {}% ***".format(
        numpy.mean(profits), numpy.mean(potentials)
    ))
    return numpy.mean(profits), numpy.mean(potentials)

def report_short_all(year):
    profits = []
    potentials = []
    data = get_short_stock_data(year)
    for row in data.query_data:
        profit = round(
            (row.price - row.year_end_price) * 100 / row.price
            ,1)
        potential = round(
            (row.price - row.year_min_price) * 100 / row.price
            ,1)
        profits.append(profit)
        potentials.append(potential)

        report_statement = REPORT.format(
            score=row.dcf_ai_score,
            symbol = row.symbol,
            exchange = row.exchangesymbol,
            start_price = row.price,
            start_date = row.pricing_date,
            end_date=row.end_pricing_date,
            end_price=row.year_end_price,
            min_price=row.year_min_price,
            max_price=row.year_max_price,
            perf_profit=profit,
            perf_potential=potential,
            avg_growth = row.average_growth_rate,
            is_consistent = row.is_yearly_consistent_rate,
            adj_growth = row.adjusted_growth,
            dcf=row.dcf,
            market_dcf=row.market_dcf,
            adj_dcf=row.adjusted_dcf,
            dcf_to_price =row.adjusted_dcf_to_price




        )

        print(report_statement)

    print("*** Average Profit: {}% , Potential: {}% ***".format(
        numpy.mean(profits), numpy.mean(potentials)
    ))
    return numpy.mean(profits), numpy.mean(potentials)


# Including Pennies Stock - 2019

In [9]:
report_all(2019)


**********- Score: -24.700000762939453 - TWMC - NasdaqCM: 2.01 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 6.92 - min: 1.92 - max: 7.745
Profit: 244.3 - max potential: 285.3 
Average growth rate: -0.419, consistent: 0, project growth: -0.805
DCF: 55443.3, market_dcf: 15135.4, adjusted_dcf: 184538.2, dcf_to_price : 9180904.98



**********- Score: 31.799999237060547 - OPGN - NasdaqCM: 1.13 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 2.01 - min: 1.13 - max: 4.04
Profit: 77.9 - max potential: 257.5 
Average growth rate: -0.063, consistent: 0, project growth: -0.125
DCF: 1729.1, market_dcf: 2006.5, adjusted_dcf: 2294.6, dcf_to_price : 202961.95



**********- Score: 19.600000381469727 - GURE - NasdaqGS: 2.5505 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 5.81 - min: 2.5505 - max: 5.89
Profit: 127.8 - max potential: 130.9 
Average growth rate: -0.63, consistent: 0, project growth: -0.893
DCF: 1765.8, m

(8.446666666666669, 67.92)

# 2018

In [39]:
report_all(2009)


**********- Score: 51.900001525878906 - FCGD - OTCPK: 80.0 on 2009-12-31 00:00:00 - *********
End date: 2010-12-31 00:00:00 - price: 197.5 - min: 60.0 - max: 225.0
Profit: 146.9 - max potential: 181.2 
Average growth rate: 2796.298, consistent: 0, project growth: 1677.701
DCF: 2.3163516406056965e+18, market_dcf: -282.9, adjusted_dcf: 1.797547401802748e+17, dcf_to_price : 2.246934252253434e+17



**********- Score: -56.0 - TACI - OTCPK: 45.00045 on 2009-12-31 00:00:00 - *********
End date: 2010-12-31 00:00:00 - price: 16.500165 - min: 15.00015 - max: 135.00135
Profit: -63.3 - max potential: 200.0 
Average growth rate: 32.061, consistent: 0, project growth: 19.547
DCF: 999577579.2, market_dcf: 6.1, adjusted_dcf: 92062178.6, dcf_to_price : 204580473.31



**********- Score: 19.100000381469727 - LCII - NYSE: 20.65 on 2009-12-31 00:00:00 - *********
End date: 2010-12-31 00:00:00 - price: 22.72 - min: 18.06 - max: 27.45
Profit: 10.0 - max potential: 32.9 
Average growth rate: 45.493, consis

(33.836666666666666, 80.43666666666667)

In [10]:
initial_investment = 1000
potential_gain = initial_investment
yearly_cumulated_gain = initial_investment
potential_yearly_gains = []
profit_yearly_gains = []

for year in range(2010, 2020):
    print (year)
    profit_per, potential_per = report_all(year)
    profit_yearly_gains.append(profit_per)
    potential_yearly_gains.append(potential_per)
    
    yearly_cumulated_gain = yearly_cumulated_gain * (1 + profit_per/100)
    potential_gain = potential_gain *(1 + potential_per/100)



2010

**********- Score: -20.299999237060547 - FSS - NYSE: 6.86 on 2010-12-31 00:00:00 - *********
End date: 2011-12-30 00:00:00 - price: 4.15 - min: 3.5 - max: 7.69
Profit: -39.5 - max potential: 12.1 
Average growth rate: 0.17, consistent: 0, project growth: 0.011
DCF: 14014475.4, market_dcf: 10054905.7, adjusted_dcf: 6747877.8, dcf_to_price : 98365465.6



**********- Score: -9.800000190734863 - DXR - AMEX: 9.25 on 2010-12-31 00:00:00 - *********
End date: 2011-12-30 00:00:00 - price: 9.1 - min: 9.08 - max: 11.17
Profit: -1.6 - max potential: 20.8 
Average growth rate: -0.464, consistent: 0, project growth: -0.659
DCF: 30813.3, market_dcf: 7200.5, adjusted_dcf: 57520.6, dcf_to_price : 621744.32



**********- Score: -25.100000381469727 - TLGT - NasdaqGS: 16.8 on 2010-12-31 00:00:00 - *********
End date: 2011-12-30 00:00:00 - price: 11.5 - min: 9.999 - max: 18.0
Profit: -31.5 - max potential: 7.1 
Average growth rate: -0.28, consistent: 0, project growth: -0.559
DCF: 11166.4, market_


**********- Score: -1.0 - LUNA - NasdaqCM: 1.4 on 2013-12-31 00:00:00 - *********
End date: 2014-12-31 00:00:00 - price: 1.42 - min: 1.23 - max: 2.5
Profit: 1.4 - max potential: 78.6 
Average growth rate: -0.565, consistent: 0, project growth: -1.122
DCF: 430.0, market_dcf: 1.2, adjusted_dcf: 1965.9, dcf_to_price : 140321.43



**********- Score: 26.399999618530273 - TCI - NYSE: 9.4399 on 2013-12-31 00:00:00 - *********
End date: 2014-12-31 00:00:00 - price: 10.3 - min: 8.8 - max: 16.98
Profit: 9.1 - max potential: 79.9 
Average growth rate: -0.363, consistent: 0, project growth: -0.658
DCF: 1199.6, market_dcf: 403.5, adjusted_dcf: 3187.6, dcf_to_price : 33667.31



**********- Score: -0.800000011920929 - NSSC - NasdaqGS: 6.28 on 2013-12-31 00:00:00 - *********
End date: 2014-12-31 00:00:00 - price: 4.7 - min: 4.07 - max: 7.57
Profit: -25.2 - max potential: 20.5 
Average growth rate: -0.162, consistent: 0, project growth: -0.294
DCF: 1052.6, market_dcf: 7.9, adjusted_dcf: 1798.1, dcf_


**********- Score: -28.700000762939453 - APWC - NasdaqGM: 2.75 on 2017-12-31 00:00:00 - *********
End date: 2018-12-31 00:00:00 - price: 1.94 - min: 1.8 - max: 2.85
Profit: -29.5 - max potential: 3.6 
Average growth rate: -0.855, consistent: 0, project growth: -1.503
DCF: 55562.7, market_dcf: 3985.5, adjusted_dcf: 248454.7, dcf_to_price : 9034616.36



**********- Score: 16.299999237060547 - PRPH - NasdaqCM: 2.17 on 2017-12-31 00:00:00 - *********
End date: 2018-12-31 00:00:00 - price: 3.15 - min: 2.24 - max: 5.41
Profit: 45.2 - max potential: 149.3 
Average growth rate: -0.547, consistent: 0, project growth: -1.086
DCF: 4760.7, market_dcf: 844.5, adjusted_dcf: 21214.6, dcf_to_price : 977531.34



**********- Score: -41.70000076293945 - DRIO - NasdaqCM: 32.04 on 2017-12-31 00:00:00 - *********
End date: 2018-12-31 00:00:00 - price: 15.52 - min: 13.2 - max: 37.2
Profit: -51.6 - max potential: 16.1 
Average growth rate: -0.087, consistent: 0, project growth: -0.169
DCF: 149690.1, market

In [11]:
print("""
Initial Investment: ${}
Average yearly gain : {},
Average potential gain: {}
Cumulated potential capital: ${}
Cumulated capital yearly end: ${}

""".format(
    initial_investment,
    round(numpy.mean(profit_yearly_gains)),
    round(numpy.mean(potential_yearly_gains)),
    '{:0,.2f}'.format(yearly_cumulated_gain),
    '{:0,.2f}'.format(potential_gain),
    
))


Initial Investment: $1000
Average yearly gain : 12.0,
Average potential gain: 63.0
Cumulated potential capital: $2,691.91
Cumulated capital yearly end: $118,314.21




# Shorting - current year 2019

In [12]:
report_short_all(2019)


**********- Score: -45.599998474121094 - DCTH - NasdaqCM: 21.25 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 8.9 - min: 7.31 - max: 21.25
Profit: 58.1 - max potential: 65.6 
Average growth rate: -0.087, consistent: 0, project growth: -0.175
DCF: -33564.8, market_dcf: -34731.2, adjusted_dcf: -49036.5, dcf_to_price : -230860.0



**********- Score: -32.70000076293945 - ARGO - NYSE: 65.75 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 33.35 - min: 26.12 - max: 70.11
Profit: 49.3 - max potential: 60.3 
Average growth rate: -1.032, consistent: 0, project growth: -1.407
DCF: -26553.6, market_dcf: -1182.2, adjusted_dcf: -62070.7, dcf_to_price : -94504.11



**********- Score: -43.70000076293945 - UFCS - NasdaqGS: 43.73 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 27.51 - min: 23.2 - max: 46.73
Profit: 37.1 - max potential: 46.9 
Average growth rate: -0.828, consistent: 0, project growth: -1.393
DCF: -10169.1

(24.24166666666666, 48.00833333333333)

In [14]:
report_short_all(2018)


**********- Score: -54.70000076293945 - INPX - NasdaqCM: 143.550143 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 4.905004 - min: 1.710001 - max: 193.500193
Profit: 96.6 - max potential: 98.8 
Average growth rate: -0.63, consistent: 0, project growth: -0.954
DCF: -596581.0, market_dcf: -81276.8, adjusted_dcf: -1476169.7, dcf_to_price : -1028430.36



**********- Score: -65.0 - TOPS - NasdaqCM: 16.462 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 0.8 - min: 0.6525 - max: 22.0
Profit: 95.1 - max potential: 96.0 
Average growth rate: 0.047, consistent: 0, project growth: 0.006
DCF: -19358.3, market_dcf: -24148.9, adjusted_dcf: -15825.7, dcf_to_price : -96234.73



**********- Score: -41.79999923706055 - SBOW - NYSE: 23.64 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 9.9 - min: 7.43 - max: 25.54
Profit: 58.1 - max potential: 68.6 
Average growth rate: -0.192, consistent: 0, project growth: -0.378
DCF: -60

(43.18888888888889, 59.22222222222222)

In [16]:
# !ipython nbconvert --to html Reporting_Rank3_2020_06_16_adjust_dcf_to_market.ipynb

[TerminalIPythonApp] WARNING | Subcommand `ipython nbconvert` is deprecated and will be removed in future versions.
[TerminalIPythonApp] WARNING | You likely want to use `jupyter nbconvert` in the future
[NbConvertApp] Converting notebook Reporting_Rank3_2020_06_16_adjust_dcf_to_market.ipynb to html
[NbConvertApp] Writing 462717 bytes to Reporting_Rank3_2020_06_16_adjust_dcf_to_market.html
